In [1]:
!pip install einops

%load_ext autoreload
%autoreload 2

import os
import sys
import glob
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torchvision as tv
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [11]:
root_dir = '/home/ubuntu'
data_dir = os.path.join(root_dir, 'CheXpert-v1.0-small')
vit_dir = os.path.join(root_dir, 'vit-pytorch')
os.chdir(vit_dir)
print(f'\nChanged CWD to "{vit_dir}"')
sys.path.append('./vit_pytorch')
from vit_pytorch import ViT


Changed CWD to "/home/ubuntu/vit-pytorch"


In [10]:
print(sys.path)

['/home/ubuntu/vit-pytorch', '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python37.zip', '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7', '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/lib-dynload', '', '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages', '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/locket-0.2.1-py3.7.egg', '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/IPython/extensions', '/home/ubuntu/.ipython', './vit_pytorch', './chexpert']


In [12]:
import pandas as pd
from PIL import Image


USE_GPU = torch.cuda.is_available()
device = 'cuda' if USE_GPU else 'cpu'

long_type = torch.cuda.LongTensor if USE_GPU else torch.long

# def read_csv(path):
#   with open(path) as f:
#     df = pd.read_csv(f)
#   return df

class CxrDataset(Dataset):
  def __init__(self,
               root_dir,
               csv_path,
               transforms):
    """
    Args:
      project_root_dir: Absolute path to the project directory.
      csv_path: Relative path to data CSV from project_root_dir.
    """
    self.root_dir = root_dir
    raw_df = pd.read_csv(os.path.join(root_dir, csv_path))
    self.df = raw_df.replace(to_replace=-1, value=1).fillna(0)
    self.transforms = transforms

  def __len__(self):
    # return len(df)
    return 64

  def __getitem__(self, idx):
    row = self.df.iloc[idx]
    image = Image.open(os.path.join(root_dir, row.Path))
    image = self.transforms(image)
    label = row['No Finding': 'Support Devices'].values.astype(np.float32)
    # TODO(sahark): Make multi-label, multi-class
    # one_hot_label = F.one_hot(label, num_classes=2).type(torch.float32)
    # torch.FloatTensor(label)
    return image, torch.from_numpy(label)

config = {
    'image_size': 256,
    'num_classes': 14,
}

train_transforms = tv.transforms.Compose(
    [
        tv.transforms.Resize((config['image_size'], config['image_size'])),
        tv.transforms.RandomHorizontalFlip(),
        tv.transforms.ToTensor(),
    ]
)

tune_csv_path = 'CheXpert-v1.0-small/valid.csv'
train_csv_path = 'CheXpert-v1.0-small/train.csv'
train_dataset = CxrDataset(root_dir, train_csv_path, train_transforms)
tune_dataset = CxrDataset(root_dir, tune_csv_path, train_transforms)    

In [13]:
vit_model = ViT(
  image_size = 256,
  patch_size = 16,
  num_classes = config['num_classes'],
  dim = 128,
  depth = 4,
  heads = 8,
  channels = 1,
  mlp_dim = 3,
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(256*256, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, config['num_classes']),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

vanilla_model = NeuralNetwork()

In [16]:
lr = 3e-5
gamma = 0.7
# https://drive.google.com/corp/drive/u/0/folders/1N5Iir5AEKZO2Lev4nOg0qi96kK9QQrKW
import math

class Trainer():
  def __init__(self,
               model,
               train_dataset,
               tune_dataset,
               **kwargs):
    self.batch_size = kwargs.pop("batch_size", 16)
    self.num_epochs = kwargs.pop("num_epochs", 1)
    self.lr = kwargs.pop("lr")
    self.device = kwargs.pop("device")

    self.train_loader = DataLoader(train_dataset, self.batch_size, shuffle=True)
    self.tune_loader = DataLoader(tune_dataset, self.batch_size * 2)
    self.model = model.to(self.device)

    self.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    self.scheduler = torch.optim.lr_scheduler.StepLR(
        self.optimizer, step_size=1, gamma=kwargs.pop("gamma"))
    self.loss_fn = torch.nn.BCEWithLogitsLoss()


  def _step(self, image, label):
    image = image.to(self.device)
    label = label.to(self.device)

    model = self.model.to(device)
    predictions = model(image)
    loss = self.loss_fn(predictions, label)

    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

  def eval_epoch(self):
    eval_loss = 0
    with torch.no_grad():
      for image, label in self.tune_loader:
        image = image.to(device)
        label = label.to(device)
        
        predictions = self.model(image)
        eval_loss += self.loss_fn(predictions, label)
    eval_loss /= self.tune_loader.__len__()
    return eval_loss
    
      
  def train(self):
    for epoch in range(self.num_epochs):
      for image, label in tqdm(self.train_loader):
        self._step(image, label)
      eval_loss = self.eval_epoch()
      print(f'epoch {epoch} eval_loss {eval_loss:.4f}')


trainer = Trainer(
    model=vit_model,
    train_dataset=train_dataset,
    tune_dataset=tune_dataset,
    num_epochs=500,
    batch_size=64,
    lr=lr,
    gamma=gamma,
    device=device,
)
trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]

epoch 0 eval_loss 0.4072


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 1 eval_loss 0.4070


  0%|          | 0/1 [00:00<?, ?it/s]

...

epoch 497 eval_loss 0.4316


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 498 eval_loss 0.4374


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 499 eval_loss 0.4355
